In [1]:
import pandas as pd
import plotly.graph_objects as go
import yfinance as yf
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv('FinVizData.csv')
# df.head()

In [3]:
symbol_list = df['Ticker'].tolist()

In [4]:
price_df = df[['Ticker', 'Price']].sort_values(by='Price', ascending=True)

In [5]:
# price_df.dtypes

In [6]:
# price_df.head(10)

In [7]:
# Candlestick plot function
def plot_stock_candlestick(symbol, months=3):
    """
    Plot candlestick chart for any stock
    
    Args:
        symbol: Stock ticker symbol (e.g., 'AAPL', 'TSLA')
        months: Number of months of historical data (default: 3)
"""

# Get data
    end_date = datetime.now()
    start_date = end_date - timedelta(days=months*30)

    df = yf.download(symbol, start=start_date, end=end_date, progress=False, auto_adjust=True)
    ticker = yf.Ticker(symbol)
    company_name = ticker.info.get('longName', symbol)

# Process data
    df.reset_index(inplace=True)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

# Create figure
    fig = go.Figure(data=[go.Candlestick(
        x=df['Date'],
        open=df['Open'].squeeze(),
        high=df['High'].squeeze(),
        low=df['Low'].squeeze(),
        close=df['Close'].squeeze()
    )])

# Calculate stats
    current_price = df['Close'].iloc[-1]
    first_price = df['Close'].iloc[0]
    price_change = current_price - first_price
    percent_change = (price_change / first_price) * 100

# Update layout
    fig.update_layout(
        title=f'{company_name} ({symbol}) - Last {months} Months<br>' +
              f'<sub>Current: ${current_price:.2f} | Change: ${price_change:.2f} ({percent_change:+.2f}%)</sub>',
        yaxis_title=f'{symbol} Stock Price (USD)',
        xaxis_title='Date',
        hovermode='x unified'
    )

    return fig

In [8]:
# fig = plot_stock_candlestick('NXDR')
# fig.show()

In [9]:
# engulfing pattern detection function

def Revsignal1(df1):
    """
    Detect engulfing candlestick patterns
    Returns: 0 = no pattern, 1 = bearish engulfing, 2 = bullish engulfing
    """
    length = len(df1)
    high = list(df1['High'])
    low = list(df1['Low'])
    close = list(df1['Close'])
    open_price = list(df1['Open'])
    signal = [0]*length
    bodydiff = [0]*length
    
    # Minimum body size threshold (0.3% of price)
    bodydiffmin = 0.003

    for row in range(1, length):
        bodydiff[row] = abs(open_price[row] - close[row])
        bodydiff[row-1] = abs(open_price[row-1] - close[row-1])
        
        # Bearish Engulfing Pattern:
        # - Previous candle is bullish (green)
        # - Current candle is bearish (red)
        # - Current open >= previous close
        # - Current close <= previous open
        if (bodydiff[row] > bodydiffmin and bodydiff[row-1] > bodydiffmin and 
            open_price[row-1] < close[row-1] and  # Previous candle is bullish
            open_price[row] > close[row] and      # Current candle is bearish
            open_price[row] >= close[row-1] and   # Current open >= previous close
            close[row] <= open_price[row-1]):     # Current close <= previous open
            signal[row] = 1  # Bearish engulfing
            
        # Bullish Engulfing Pattern:
        # - Previous candle is bearish (red)
        # - Current candle is bullish (green)
        # - Current open <= previous close
        # - Current close >= previous open
        elif (bodydiff[row] > bodydiffmin and bodydiff[row-1] > bodydiffmin and
            open_price[row-1] > close[row-1] and  # Previous candle is bearish
            open_price[row] < close[row] and      # Current candle is bullish
            open_price[row] <= close[row-1] and   # Current open <= previous close
            close[row] >= open_price[row-1]):     # Current close >= previous open
            signal[row] = 2  # Bullish engulfing
        else:
            signal[row] = 0
            
    return signal


In [10]:
# Function to analyze a ticker for engulfing patterns
def analyze_ticker_patterns(symbol, days=90):
    """
    Download data for a ticker and detect engulfing patterns
    
    Args:
        symbol: Stock ticker symbol
        days: Number of days of historical data (default: 90)
    
    Returns:
        Dictionary with pattern analysis results
    """
    try:
        # Download data
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        
        ticker_df = yf.download(symbol, start=start_date, end=end_date, progress=False, auto_adjust=True)
        
        if ticker_df.empty:
            return None
            
        # Reset index to make Date a column
        ticker_df.reset_index(inplace=True)
        
        # Handle MultiIndex columns if present
        if isinstance(ticker_df.columns, pd.MultiIndex):
            ticker_df.columns = ticker_df.columns.get_level_values(0)
        
        # Apply pattern detection
        ticker_df['Signal'] = Revsignal1(ticker_df)
        
        # Get most recent signal
        latest_signal = ticker_df['Signal'].iloc[-1]
        latest_date = ticker_df['Date'].iloc[-1]
        
        # Count patterns in the period
        bearish_count = (ticker_df['Signal'] == 1).sum()
        bullish_count = (ticker_df['Signal'] == 2).sum()
        
        # Get latest price info
        latest_close = ticker_df['Close'].iloc[-1]
        
        return {
            'Ticker': symbol,
            'Latest_Signal': latest_signal,
            'Latest_Date': latest_date,
            'Bearish_Count': bearish_count,
            'Bullish_Count': bullish_count,
            'Latest_Close': latest_close,
            'Data': ticker_df
        }
        
    except Exception as e:
        print(f"Error analyzing {symbol}: {str(e)}")
        return None


In [11]:
# Analyze all tickers from FinViz data
print(f"Analyzing {len(symbol_list)} tickers for engulfing patterns...")

results = []
for i, symbol in enumerate(symbol_list, 1):
    print(f"Processing {i}/{len(symbol_list)}: {symbol}", end='\r')
    result = analyze_ticker_patterns(symbol, days=90)  # Using 90 days for more data
    if result:
        results.append({
            'Ticker': result['Ticker'],
            'Latest_Signal': result['Latest_Signal'],
            'Latest_Signal_Name': {0: 'None', 1: 'Bearish', 2: 'Bullish'}[result['Latest_Signal']],
            'Latest_Date': result['Latest_Date'],
            'Bearish_Count_90d': result['Bearish_Count'],
            'Bullish_Count_90d': result['Bullish_Count'],
            'Latest_Close': result['Latest_Close']
        })
    # Small delay to avoid rate limiting
    import time
    time.sleep(0.1)

print(f"\nCompleted analysis of {len(results)} tickers!")

# Create summary DataFrame
pattern_df = pd.DataFrame(results).sort_values(['Latest_Signal', 'Latest_Close'], ascending=False)
pattern_df


Analyzing 79 tickers for engulfing patterns...
Processing 79/79: QTRXK
Completed analysis of 79 tickers!


,Ticker,Latest_Signal,Latest_Signal_Name,Latest_Date,Bearish_Count_90d,Bullish_Count_90d,Latest_Close
37,TCMD,2,Bullish,2025-11-21,1,9,25.700001
51,MEC,2,Bullish,2025-11-21,4,7,15.610000
33,ALMS,2,Bullish,2025-11-21,6,3,7.300000
78,QTRX,2,Bullish,2025-11-21,5,3,6.660000
36,NXDR,2,Bullish,2025-11-21,5,2,1.690000
...,...,...,...,...,...,...,...
48,WALD,0,None,2025-11-21,6,4,3.080000
32,BLND,0,None,2025-11-21,2,6,3.010000
68,BFLY,0,None,2025-11-21,3,3,2.650000
35,PACB,0,None,2025-11-21,6,4,2.220000


In [12]:
# Filter tickers with recent bearish engulfing patterns
bearish_signals = pattern_df[pattern_df['Latest_Signal'] == 1].sort_values('Latest_Date', ascending=False)
print(f"\n🔴 Tickers with BEARISH engulfing pattern (most recent):")
print(f"Found {len(bearish_signals)} tickers\n")
bearish_signals



🔴 Tickers with BEARISH engulfing pattern (most recent):
Found 1 tickers



,Ticker,Latest_Signal,Latest_Signal_Name,Latest_Date,Bearish_Count_90d,Bullish_Count_90d,Latest_Close
73,MLAB,1,Bearish,2025-11-21,4,5,70.599998


In [13]:
# Filter tickers with recent bullish engulfing patterns
bullish_signals = pattern_df[pattern_df['Latest_Signal'] == 2].sort_values('Latest_Date', ascending=False)
print(f"\n🟢 Tickers with BULLISH engulfing pattern (most recent):")
print(f"Found {len(bullish_signals)} tickers\n")
bullish_signals



🟢 Tickers with BULLISH engulfing pattern (most recent):
Found 6 tickers



,Ticker,Latest_Signal,Latest_Signal_Name,Latest_Date,Bearish_Count_90d,Bullish_Count_90d,Latest_Close
37,TCMD,2,Bullish,2025-11-21,1,9,25.700001
51,MEC,2,Bullish,2025-11-21,4,7,15.610000
33,ALMS,2,Bullish,2025-11-21,6,3,7.300000
78,QTRX,2,Bullish,2025-11-21,5,3,6.660000
36,NXDR,2,Bullish,2025-11-21,5,2,1.690000
49,NNDM,2,Bullish,2025-11-21,5,6,1.580000


In [14]:
fig = plot_stock_candlestick('NNDM')
fig.show()

In [15]:
# Merge pattern data with original FinViz data
merged_df = df.merge(
    pattern_df[['Ticker', 'Latest_Signal_Name', 'Latest_Close', 'Bearish_Count_90d', 'Bullish_Count_90d']], 
    on='Ticker', 
    how='left'
)

# Save to CSV
merged_df.to_csv('FinVizData_with_patterns.csv', index=False)
print("✅ Merged data saved to 'FinVizData_with_patterns.csv'")

# merged_df.head()


✅ Merged data saved to 'FinVizData_with_patterns.csv'


In [16]:
# merged_df.columns

In [17]:
# Example: Visualize a ticker with detected patterns
# Pick the first ticker with a bullish or bearish signal
example_ticker = None

if len(bullish_signals) > 0:
    example_ticker = bullish_signals.iloc[0]['Ticker']
    pattern_type = "BULLISH"
elif len(bearish_signals) > 0:
    example_ticker = bearish_signals.iloc[0]['Ticker']
    pattern_type = "BEARISH"

if example_ticker:
    print(f"Visualizing {example_ticker} with {pattern_type} engulfing pattern:\n")
    fig = plot_stock_candlestick(example_ticker, months=3)
    fig.show()
else:
    print("No engulfing patterns detected in the dataset.")


Visualizing TCMD with BULLISH engulfing pattern:

